This notebook contains an example for teaching.

## Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question,
but we could begin to investigate from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender. Two main questions here are:


* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data


The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015.  We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors;  individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$. 

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size $n=5150$.

## Data analysis

We start by loading the data set.

In [1]:
load("../data/wage2015_subsample_inference.Rdata")
dim(data)

[1] 5150   20

Let's have a look at the structure of the data.

In [2]:
str(data)

'data.frame':	5150 obs. of  20 variables:
 $ wage : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage: num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex  : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg  : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg  : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2 : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3 : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4 : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 269 23 99 86 226 232 184 146 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 19 1 6 5 17 17 13 10 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 12 165 231 176 171 135 210 201 ...
 $ ind2 : Factor w/ 

We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [3]:
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]

cat("Number of observation:", n, '\n')
cat( "Number of raw regressors:", p)

Number of observation: 5150 
Number of raw regressors: 18

For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [4]:
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")

In [6]:
library(xtable)
Z_subset <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"))]
table <- matrix(0, 12, 1)
table[1:12,1]   <- as.numeric(lapply(Z_subset,mean))
rownames(table) <- c("Log Wage","Sex","Some High School","High School Graduate","Some College","College Graduate", "Advanced Degree","Midwest","South","West","Northeast","Experience")
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

,Sample mean
,<dbl>
Log Wage,2.97078670
Sex,0.44446602
Some High School,0.02330097
High School Graduate,0.24388350
Some College,0.27805825
College Graduate,0.31766990
Advanced Degree,0.13708738
Midwest,0.25961165
South,0.29650485


E.g., the share of female workers in our sample is ~44% ($sex=1$ if female).

Alternatively, we can also print the table as latex.

In [7]:
print(tab, type="latex")

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% 
\begin{tabular}{rr}
  \hline
 & Sample mean \\ 
  \hline
Log Wage & 2.97 \\ 
  Sex & 0.44 \\ 
  Some High School & 0.02 \\ 
  High School Graduate & 0.24 \\ 
  Some College & 0.28 \\ 
  College Graduate & 0.32 \\ 
  Advanced Degree & 0.14 \\ 
  Midwest & 0.26 \\ 
  South & 0.30 \\ 
  West & 0.22 \\ 
  Northeast & 0.23 \\ 
  Experience & 13.76 \\ 
   \hline
\end{tabular}


## Prediction Question

Now, we will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

\begin{equation}\label{decompose}
Y = \beta'X+ \epsilon.
\end{equation}

Our goals are

* Predict wages  using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators, regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is *experience* times the indicator of having a *college degree*.

Using the **Flexible Model**, enables us to approximate the real relationship by a
 more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [8]:
# 1. basic model
basic <- lwage ~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2 + ind2)
regbasic <- lm(basic, data=data)
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model



Call:
lm(formula = basic, data = data)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   3.722235    -0.072857     0.008568    -0.592798    -0.504337    -0.411994  
        clg           mw           so           we        occ22        occ23  
  -0.182216    -0.027541    -0.034454     0.017249    -0.076472    -0.034678  
      occ24        occ25        occ26        occ27        occ28        occ29  
  -0.096202    -0.187915    -0.414933    -0.045987    -0.377847    -0.215752  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.010623    -0.455834    -0.307589    -0.361440    -0.499495    -0.464482  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.233715    -0.412588    -0.340418    -0.241480    -0.212628    -0.288413  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.422394    -0.116836    -0.244493    -0.273533    -0.249368    -0.139588

Number of regressors in the basic model: 51 


##### Note that the basic model consists of $51$ regressors.

In [9]:
# 2. flexible model
flex <- lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)
regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length(regflex$coef)) # number of regressors in the Flexible Model


Call:
lm(formula = flex, data = data)

Coefficients:
(Intercept)          sex          shs          hsg          scl          clg  
  3.8602606   -0.0695532   -0.1233089   -0.5289024   -0.2920581   -0.0411641  
      occ22        occ23        occ24        occ25        occ26        occ27  
  0.1613397    0.2101514    0.0708570   -0.3960076   -0.2310611    0.3147249  
      occ28        occ29       occ210       occ211       occ212       occ213  
 -0.1875417   -0.3390270    0.0209545   -0.6424177   -0.0674774   -0.2329781  
     occ214       occ215       occ216       occ217       occ218       occ219  
  0.2562009   -0.1938585   -0.0551256   -0.4156093   -0.4822168   -0.2579412  
     occ220       occ221       occ222        ind23        ind24        ind25  
 -0.3010203   -0.4271811   -0.8694527   -1.2473654   -0.0948281   -0.5293860  
      ind26        ind27        ind28        ind29       ind210       ind211  
 -0.6221688   -0.5047497   -0.7295442   -0.8025334   -0.5805840   -0.9852350 

Number of regressors in the flexible model: 246

Note that the flexible model consists of $246$ regressors.

Try Lasso next

In [10]:
library(hdm)
flex <- lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)
lassoreg<- rlasso(flex, data=data)

sumlasso<- summary(lassoreg)


Call:
rlasso.formula(formula = flex, data = data)

Post-Lasso Estimation:  TRUE 

Total number of variables: 245
Number of selected variables: 24 

Residuals: 
     Min       1Q   Median       3Q      Max 
-2.03159 -0.29132 -0.01137  0.28472  3.63651 

            Estimate
(Intercept)    3.137
sex            0.000
shs           -0.480
hsg           -0.404
scl           -0.306
clg            0.000
occ22          0.041
occ23          0.109
occ24          0.000
occ25          0.000
occ26         -0.327
occ27          0.000
occ28         -0.257
occ29          0.000
occ210         0.038
occ211        -0.398
occ212         0.000
occ213        -0.131
occ214        -0.189
occ215        -0.387
occ216         0.000
occ217        -0.280
occ218         0.000
occ219         0.000
occ220         0.000
occ221         0.000
occ222        -0.209
ind23          0.000
ind24          0.000
ind25          0.000
ind26          0.000
ind27          0.000
ind28          0.000
ind29         -0.169
ind210     

Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [11]:
# Assess the predictive performance

sumbasic <- summary(regbasic)
sumflex <- summary(regflex)

#  R-squared 
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
R2.adj1 <- sumbasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adj1, "\n")

R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
R2.adj2 <- sumflex$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adj2, "\n")

R2.L <- sumlasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.L, "\n")
R2.adjL <- sumlasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjL, "\n")

R-squared for the basic model:  0.3100465 
adjusted R-squared for the basic model:  0.3032809 
R-squared for the flexible model:  0.3511099 
adjusted R-squared for the flexible model:  0.3186919 
R-squared for the lasso with flexible model:  0.2778653 
adjusted R-squared for the flexible model:  0.2744836 


In [12]:
# calculating the MSE
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")
p1 <- sumbasic$df[1] # number of regressors
MSE.adj1 <- (n/(n-p1))*MSE1
cat("adjusted MSE for the basic model: ", MSE.adj1, "\n")

MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")
p2 <- sumflex$df[1]
MSE.adj2 <- (n/(n-p2))*MSE2
cat("adjusted MSE for the flexible model: ", MSE.adj2, "\n")


MSEL <-mean(sumlasso$res^2)
cat("MSE for the lasso flexible model: ", MSEL, "\n")
pL <- length(sumlasso$coef)
MSE.adjL <- (n/(n-pL))*MSEL
cat("adjusted MSE for the lasso flexible model: ", MSE.adjL, "\n")

MSE for the basic model:  0.2244251 
adjusted MSE for the basic model:  0.2266697 
MSE for the flexible model:  0.2110681 
adjusted MSE for the flexible model:  0.221656 
MSE for the lasso flexible model:  0.2348928 
adjusted MSE for the lasso flexible model:  0.2466758 


In [13]:
library(xtable)
table <- matrix(0, 3, 5)
table[1,1:5]   <- c(p1,R2.1,MSE1,R2.adj1,MSE.adj1)
table[2,1:5]   <- c(p2,R2.2,MSE2,R2.adj2,MSE.adj2)
table[3,1:5]   <- c(pL,R2.L,MSEL,R2.adjL,MSE.adjL)
colnames(table)<- c("p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$")
rownames(table)<- c("basic reg","flexible reg", "lasso flex")
tab<- xtable(table, digits =c(0,0,2,2,2,2))
print(tab,type="latex") # type="latex" for printing table in LaTeX
tab

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% 
\begin{tabular}{rrrrrr}
  \hline
 & p & \$R\verb|^|2\_\{sample\}\$ & \$MSE\_\{sample\}\$ & \$R\verb|^|2\_\{adjusted\}\$ & \$MSE\_\{adjusted\}\$ \\ 
  \hline
basic reg & 51 & 0.31 & 0.22 & 0.30 & 0.23 \\ 
  flexible reg & 246 & 0.35 & 0.21 & 0.32 & 0.22 \\ 
  lasso flex & 246 & 0.28 & 0.23 & 0.27 & 0.25 \\ 
   \hline
\end{tabular}


,p,$R^2_{sample}$,$MSE_{sample}$,$R^2_{adjusted}$,$MSE_{adjusted}$
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
basic reg,51,0.3100465,0.2244251,0.3032809,0.2266697
flexible reg,246,0.3511099,0.2110681,0.3186919,0.2216560
lasso flex,246,0.2778653,0.2348928,0.2744836,0.2466758


Considering all measures above, the flexible model performs slightly better than the basic model. 

One procedure to circumvent this issue is to use **data splitting** that is described and applied in the following.

## Data Splitting

Measure the prediction quality of the two models via data splitting:

- Randomly split the data into one training sample and one testing sample. Here we just use a simple method (stratified splitting is a more sophiscticated version of splitting that we can consider).
- Use the training sample for estimating the parameters of the Basic Model and the Flexible Model.
- Use the testing sample for evaluation. Predict the $\mathtt{wage}$  of every observation in the testing sample based on the estimated parameters in the training sample.
- Calculate the Mean Squared Prediction Error $MSE_{test}$ based on the testing sample for both prediction models. 

In [14]:
n

[1] 5150

In [34]:
#splitting the data

set.seed(1) # to make the results replicable (generating random numbers)
random <- sample(1:n, floor(n*1/2))
length(random)

[1] 2575

In [35]:
# draw (4/5)*n random numbers from 1 to n without replacing them
train <- data[random,] # training sample
test <- data[-random,] # testing sample
dim(test)

[1] 2575   20

In [36]:
basic

lwage ~ (sex + exp1 + shs + hsg + scl + clg + mw + so + we + 
    occ2 + ind2)

In [37]:
# basic model
# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)
regbasic


Call:
lm(formula = basic, data = train)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   3.563857    -0.025206     0.008506    -0.592632    -0.518732    -0.382860  
        clg           mw           so           we        occ22        occ23  
  -0.151858    -0.058771    -0.049817    -0.033036    -0.054774     0.018731  
      occ24        occ25        occ26        occ27        occ28        occ29  
  -0.082256    -0.127580    -0.496692    -0.159043    -0.378068    -0.262423  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.021736    -0.480576    -0.317988    -0.368513    -0.401788    -0.516071  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.221725    -0.403448    -0.128888    -0.237452    -0.139704    -0.281448  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.386172    -0.062550    -0.047749    -0.143264    -0.065055    -0.09602

In [38]:
# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)
trainregbasic

10       12       19       30       44       71       77       84 
2.771302 3.385309 3.059342 3.241209 2.687760 3.204623 2.704065 2.527540 
     119      128      129      135      149      152      157      160 
3.008514 2.687811 2.319409 2.479331 3.220366 3.095546 2.864941 2.934074 
     176      191      206      213      221      248      254      264 
3.349614 3.048279 2.615542 3.016152 3.073951 2.621533 2.472478 2.417316 
     276      281      291      352      367      368      376      399 
2.739061 3.135662 3.201402 2.378953 2.678110 2.601807 2.668693 3.734441 
     416      431      436      445      464      466      467      471 
2.798889 2.819468 2.947146 2.387772 2.714640 3.228808 2.979328 2.926916 
     472      475      496      540      546      567      573      574 
3.120690 2.866691 2.952359 2.767675 2.886291 2.387416 2.894797 2.934074 
     576      610      629      652      679      687      705      744 
2.940076 3.020660 3.280072 2.776290 2.421441 3.247889 2.745827 2.941559 
     759      765      769      794      800      809      827      848 
2.741153 2.789737 2.748410 3.347728 3.294960 2.957301 2.889784 2.311965 
     858      861      878      893      911      924      938      945 
2.895003 2.731736 3.103980 2.870546 3.457367 2.995252 2.523287 2.985111 
     952      953      954      955      960      977     1003     1012 
3.205297 2.774267 2.656828 2.697282 2.766139 3.401681 3.116533 3.121465 
    1018     1019     1057     1059     1065     1066     1075     1076 
3.066881 3.249081 2.606209 3.082508 2.800164 2.986147 3.462588 3.577905 
    1079     1083     1084     1101     1105     1143     1161     1176 
2.268685 3.254102 2.802528 2.613036 2.741975 2.801809 3.684926 3.434686 
    1195     1221     1223     1224     1235     1238     1271     1289 
2.547179 2.965115 2.827974 2.841465 2.879011 3.596724 2.758166 3.139259 
    1299     1310     1342     1359     1385     1387     1405     1407 
2.817710 3.091015 3.645356 2.827974 2.890973 3.439843 2.729667 2.593853 
    1432     1451     1472     1483     1491     1508     1527     1536 
2.930655 3.200816 2.972664 2.596161 2.655558 2.858866 2.408809 2.849625 
    1538     1550     1551     1566     1568     1574     1593     1614 
2.966945 2.728903 3.263529 3.525575 2.877206 3.260537 2.999161 3.438245 
    1631     1646     1647     1653     1654     1661     1669     1693 
3.323710 2.794306 3.642395 3.191297 3.039444 3.134872 2.870047 2.749176 
    1711     1715     1723     1725     1730     1733     1734     1748 
2.875878 3.026536 2.794022 3.373354 2.772198 2.716144 2.748913 2.949263 
    1749     1751     1763     1775     1782     1800     1824     1826 
2.917106 2.970587 3.466248 2.411170 2.993711 2.909397 2.964200 2.827936 
    1827     1829     1837     1861     1862     1867     1889     1896 
2.608921 2.613174 2.599531 3.653385 3.211200 2.923328 3.457324 3.101441 
    1915     1916     1917     1923     1926     1930     1931     1933 
2.661077 2.688755 3.339498 3.557528 3.320246 3.628368 3.653887 3.615622 
    1939     1945     1946     1947     1949     1964     1973     1978 
3.314292 2.909113 3.501087 3.568323 3.156871 2.568548 2.947191 3.393175 
    1982     1986     1988     1999     2020     2024     2029     2055 
3.061771 3.022714 2.875566 2.826428 2.856797 2.769904 2.909913 2.749439 
    2070     2089     2110     2122     2123     2132     2135     2143 
2.807322 3.317453 3.084866 3.112205 3.241526 2.713279 2.865287 3.347975 
    2159     2172     2194     2199     2210     2215     2224     2255 
3.295222 2.758166 3.089552 3.204938 3.594343 3.089552 3.086187 3.414793 
    2278     2298     2307     2313     2315     2324     2330     2333 
3.555927 3.230659 3.581641 3.497304 3.164123 2.947146 3.157091 3.236584 
    2352     2362     2371     2373     2374     2381     2425     2445 
2.679957 2.947728 3.069955 2.850498 3.506608 2.845733 3.306807 3.007980 
    2465     2488     2490     2494     2500     2502     2512  

In [39]:
y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)

cat("Test MSE for the basic model: ", MSE.test1, " ")

cat("Test R2 for the basic model: ", R2.test1)

Test MSE for the basic model:  0.2354335  Test R2 for the basic model:  0.2777643

In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$.

In [40]:
# flexible model
# estimating the parameters
#options(warn=-1)
regflex <- lm(flex, data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test)
y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- MSE.test2/var(y.test)

cat("Test MSE for the flexible model: ", MSE.test2, " ")

cat("Test R2 for the flexible model: ", R2.test2)

Warning message in predict.lm(regflex, newdata = test):
"prediction from a rank-deficient fit may be misleading"


Test MSE for the flexible model:  0.2803699  Test R2 for the flexible model:  0.1399136

In [41]:
length(y.test)

[1] 2575

In the flexible model, the discrepancy between the $MSE_{test}$ and the $MSE_{sample}$ is not large.

It is worth to notice that the $MSE_{test}$ vary across different data splits. Hence, it is a good idea average the out-of-sample MSE over different data splits to get valid results.

Nevertheless, we observe that, based on the out-of-sample $MSE$, the basic model using ols regression performs is about as well (or slightly better) than the flexible model. 


Next, let us use lasso regression in the flexible model instead of ols regression. Lasso (*least absolute shrinkage and selection operator*) is a penalized regression method that can be used to reduce the complexity of a regression model when the number of regressors $p$ is relatively large in relation to $n$. 

Note that the out-of-sample $MSE$ on the test sample can be computed for any other black-box prediction method as well. Thus, let us finally compare the performance of lasso regression in the flexible model to ols regression.

In [42]:
# flexible model using lasso

# estimating the parameters
library(hdm)
reglasso <- rlasso(flex, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglasso<- predict(reglasso, newdata=test)
MSE.lasso <- sum((y.test-trainreglasso)^2)/length(y.test)
R2.lasso<- 1- MSE.lasso/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE.lasso, " ")

cat("Test R2 for the lasso flexible model: ", R2.lasso)

Test MSE for the lasso on flexible model:  0.2480306  Test R2 for the lasso flexible model:  0.2391205

Finally, let us summarize the results:

In [43]:
table2 <- matrix(0, 3,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[3,1]   <- MSE.lasso
table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2
table2[3,2]   <- R2.lasso

rownames(table2)<- c("basic reg","flexible reg","lasso regression")
colnames(table2)<- c("$MSE_{test}$", "$R^2_{test}$")
tab2 <- xtable(table2, digits =3)
tab2

,$MSE_{test}$,$R^2_{test}$
,<dbl>,<dbl>
basic reg,0.2354335,0.2777643
flexible reg,0.2803699,0.1399136
lasso regression,0.2480306,0.2391205


In [41]:
print(tab2,type="latex") # type="latex" for printing table in LaTeX

% latex table generated in R 4.0.4 by xtable 1.8-4 package
% 
\begin{tabular}{rrr}
  \hline
 & \$MSE\_\{test\}\$ & \$R\verb|^|2\_\{test\}\$ \\ 
  \hline
basic reg & 0.197 & 0.328 \\ 
  flexible reg & 0.206 & 0.296 \\ 
  lasso regression & 0.213 & 0.275 \\ 
   \hline
\end{tabular}
